# Analyse av historisk værdata i Stryn


In [38]:
import pandas as pd 
import os
from pandasql import sqldf

In [46]:
df = pd.read_csv("../data/table.csv",delimiter=';')
df = df.dropna()
#df = df.drop(columns=["Nedbør (1 t)"])
print(df.head())

             Navn  Stasjon Tid(norsk normaltid) Nedbør (1 t) Lufttemperatur  \
0  Stryn - Kroken  SN58900     01.03.2024 01:00            0            7,3   
1  Stryn - Kroken  SN58900     01.03.2024 02:00            0            6,3   
2  Stryn - Kroken  SN58900     01.03.2024 03:00            0            7,1   
3  Stryn - Kroken  SN58900     01.03.2024 04:00            0            6,3   
4  Stryn - Kroken  SN58900     01.03.2024 05:00            0            5,2   

  Vindretning Middelvind  
0          84        1,6  
1          25        0,6  
2         119        1,6  
3         268        1,5  
4         224        0,4  


Under skjekker vi hvilke rader som mangler verdier, slik at vi kan fylle dem med hjelp av forskjellige metoder. Dette er en fin måte å filtrere data på og finne hull i datasettet. En god metode for å fylle tomrom rom er å ta i bruk mooving average som tar gjennomsnitte av X antal timer før og bruker det som verdi. Her ser vi at vi mangler 1 verdi for hver av kolonnene. Det viser seg å være den siste raden i csv filen vår som er en setning "Data er gyldig per 16.03.2025 (CC BY 4.0), Meteorologisk institutt (MET);;;;;;". Denne setningen har vi valgt å fjerne i kodesnuten over.

In [47]:
print(df.isnull().sum())

Navn                    0
Stasjon                 0
Tid(norsk normaltid)    0
Nedbør (1 t)            0
Lufttemperatur          0
Vindretning             0
Middelvind              0
dtype: int64


In [48]:
print(df[df.isnull().any(axis=1)])

Empty DataFrame
Columns: [Navn, Stasjon, Tid(norsk normaltid), Nedbør (1 t), Lufttemperatur, Vindretning, Middelvind]
Index: []


## For å forstå dataens struktur og innhold har vi benyttet oss av diverse funksjoner 
- Først ser vi på innholde ved hjelp av list comprehensions og trekker ut temperaturene fra DataFramen.
- Videre tar vi i bruk en iterator til å gå gjennom DataFramen.
- Til slutt bruker vi Pandasql for å filtere og analysere dataen 

In [42]:
#List 
temperatures = [temp for temp in df["Lufttemperatur"] if temp is not None]
print(temperatures[:10]) 

['7,3', '6,3', '7,1', '6,3', '5,2', '4', '3,1', '2,6', '3,3', '4,9']


In [43]:
#Iterator
for index, row in df.iterrows():
    print(f"Stasjon: {row['Stasjon']}, Tid: {row['Tid(norsk normaltid)']}, Temperatur: {row['Lufttemperatur']}°C")
    if index == 5: 
        break

    

Stasjon: SN58900, Tid: 01.03.2024 01:00, Temperatur: 7,3°C
Stasjon: SN58900, Tid: 01.03.2024 02:00, Temperatur: 6,3°C
Stasjon: SN58900, Tid: 01.03.2024 03:00, Temperatur: 7,1°C
Stasjon: SN58900, Tid: 01.03.2024 04:00, Temperatur: 6,3°C
Stasjon: SN58900, Tid: 01.03.2024 05:00, Temperatur: 5,2°C
Stasjon: SN58900, Tid: 01.03.2024 06:00, Temperatur: 4°C


In [44]:
#Pandasql
vind = "SELECT `Tid(norsk normaltid)`, Middelvind FROM df WHERE Middelvind > 5"
result = sqldf(vind)

print(result.head())

  Tid(norsk normaltid) Middelvind
0     16.11.2024 17:00        5,5
1     05.12.2024 13:00        5,4
2     15.12.2024 19:00        5,1


# OPPGAVE 3 Databehandling 

In [45]:
def load_and_clean_csv(filepath):
    # Skriver inn nye kolonnenavn
    df.columns = ["Navn", "Stasjon", "Tid", "Nedbør", "Lufttemperatur", "Vindretning", "Middelvind"]

    # Gjør om "Tid" til datetime-format
    df["Tid(norsk normaltid)"] = pd.to_datetime(df["Tid(norsk normaltid)"], format="%d.%m.%Y %H:%M", errors="coerce")
    
    # Gjør om de numeriske kolonnene til float
    num_cols = ["Nedbør", "Lufttemperatur", "Vindretning", "Middelvind"]
    df[num_cols] = df[num_cols].astype(float)

    # Fyll inn eventuelle manglende verdier 
    df.fillna(df.mean(numeric_only=True), inplace=True)

    return df

filepath = "../data/table.csv" 
df_cleaned = load_and_clean_csv(filepath)
print(df_cleaned.head())

KeyError: 'Tid(norsk normaltid)'

In [ ]:
def remove_outliers(df):
    """
    Fjerner urealistiske verdier i datasettene.
    """
    df = df[(df["Lufttemperatur"].between(-50, 50))]  # Temperatur bør være mellom -50 og 50 °C
    df = df[(df["Vindretning"].between(0, 360))]  # Vindretning må være mellom 0° og 360°
    
    return df

df_cleaned = remove_outliers(df_cleaned)
#df_cleaned.to_csv("renset_værdata.csv", index=False)


NameError: name 'df_cleaned' is not defined

In [ ]:
#Kanksje bruke
def handle_missing_data(df, method="mean"):
    """
    Fyller inn manglende verdier i numeriske kolonner.
    method: "mean" (gjennomsnitt), "median" (median), "zero" (fyll med 0)
    """
    if method == "mean":
        df.fillna(df.mean(numeric_only=True), inplace=True)
    elif method == "median":
        df.fillna(df.median(numeric_only=True), inplace=True)
    elif method == "zero":
        df.fillna(0, inplace=True)
    
    return df

df_cleaned = handle_missing_data(df_cleaned, method="median")


NameError: name 'df_cleaned' is not defined